In [1]:
import networkx as nx
import pylab as plt
import pandas as pd
import numpy as np
import math
from timeit import default_timer as timer
from matplotlib.pyplot import figure


In [2]:
PR_df = pd.read_csv('./PR_df.csv')
PR_df.sort_values(by='0',inplace = True, ascending=False)
PR_df.reset_index(drop = True, inplace = True)
PR_df.rename(columns={'Unnamed: 0':'Author', '0':'PR_values'}, inplace = True)
PR_df

,Author,PR_values
0,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,0.042414
1,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,0.015953
2,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,0.014931
3,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,0.014887
4,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,0.014411
...,...,...
873,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,0.000483
874,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,0.000483
875,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,0.000483
876,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,0.000483


In [3]:
DB_AA_originale = pd.read_csv('./DB_articles_authors_built.csv')


#genesis_tx_node_id = DB_AA_originale[DB_AA_originale['To_Author_Pub_key']=='Not_available']['ToNodeId'].values[0]
#RIMUOVO LE CITAZIONI INIZIALI FATTE VERSO LA GENESIS TX DAGLI ARTICOLI DI FRONTIERA
#DB_AA_originale = DB_AA_originale[DB_AA_originale['ToNodeId']!= genesis_tx_node_id]
DB_AA_originale

,FromNodeId,From_Author_Pub_Key,ToNodeId,To_Author_Pub_key
0,bea8b41266f68383aab0ab6f11e1423db72df15ff8357a...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,7abd48c9ff5cd7ed0f42d4879fde173481d087ea25372f...,Not_available
1,48f59e57815078456b89fe34932606fcf1e5fa7874cf00...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,7abd48c9ff5cd7ed0f42d4879fde173481d087ea25372f...,Not_available
2,e2e9da947c52e0f6107d5aa5d6e03e0e9fcafe7bccde08...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,7abd48c9ff5cd7ed0f42d4879fde173481d087ea25372f...,Not_available
3,f2b1c07718734fb4f48789faef537bc69604b53befe340...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,48f59e57815078456b89fe34932606fcf1e5fa7874cf00...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...
4,c356dd48fe8a94bc92ef446443ee88f49eb28dee5e95bd...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,7abd48c9ff5cd7ed0f42d4879fde173481d087ea25372f...,Not_available
...,...,...,...,...
2190,11be570f418641c06fe7a7bb832b237a51d3a6f1615f3a...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,1a730dd1844edc082bd852f6a8617db09885158f30e105...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...
2191,11be570f418641c06fe7a7bb832b237a51d3a6f1615f3a...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,3cbf33b87b930b9f7861e16e41b2596d98ddb08c128afb...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...
2192,11be570f418641c06fe7a7bb832b237a51d3a6f1615f3a...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,860b9434133c78fede874d0380cf50697b49af8c7c0b8d...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...
2193,11be570f418641c06fe7a7bb832b237a51d3a6f1615f3a...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,c4a161d608769cd6fe604685c9204f81351eb9f07032a7...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...


In [4]:
DB_AA_originale.FromNodeId.nunique()

1066

In [5]:
DB_AA_originale.ToNodeId.nunique()

663

In [6]:
genesis_tx_node_id = DB_AA_originale[DB_AA_originale['To_Author_Pub_key']=='Not_available']['ToNodeId'].values[0]
#RIMUOVO LE CITAZIONI INIZIALI FATTE VERSO LA GENESIS TX DAGLI ARTICOLI DI FRONTIERA
DB_AA_originale_senza_genesis = DB_AA_originale[DB_AA_originale['ToNodeId']!= genesis_tx_node_id]

res = DB_AA_originale_senza_genesis.groupby('To_Author_Pub_key')['From_Author_Pub_Key'].count()
cit_entranti_max = res.max()
cit_entranti_max


58

In [7]:
def readMsgDB():
    print('Checking for local message_id stored...\n')
    try: 
        file = open("lista_msgDB.txt", "r")
        msg_DB = file.readlines()

        for i in range(len(msg_DB)):
            msg_DB[i] = msg_DB[i].replace("\n", "")

        print(f'Found #{len(msg_DB)} messages from the file lista_msgDB.txt \n')

        file.close() 
    except:
        
        msg_DB = []

    
    return msg_DB

msg_DB = readMsgDB()

Checking for local message_id stored...

Found #1066 messages from the file lista_msgDB.txt 



In [8]:
DB_AA_originale[DB_AA_originale['FromNodeId'].isin(msg_DB)]

,FromNodeId,From_Author_Pub_Key,ToNodeId,To_Author_Pub_key
0,bea8b41266f68383aab0ab6f11e1423db72df15ff8357a...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,7abd48c9ff5cd7ed0f42d4879fde173481d087ea25372f...,Not_available
1,48f59e57815078456b89fe34932606fcf1e5fa7874cf00...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,7abd48c9ff5cd7ed0f42d4879fde173481d087ea25372f...,Not_available
2,e2e9da947c52e0f6107d5aa5d6e03e0e9fcafe7bccde08...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,7abd48c9ff5cd7ed0f42d4879fde173481d087ea25372f...,Not_available
3,f2b1c07718734fb4f48789faef537bc69604b53befe340...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,48f59e57815078456b89fe34932606fcf1e5fa7874cf00...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...
4,c356dd48fe8a94bc92ef446443ee88f49eb28dee5e95bd...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,7abd48c9ff5cd7ed0f42d4879fde173481d087ea25372f...,Not_available
...,...,...,...,...
2190,11be570f418641c06fe7a7bb832b237a51d3a6f1615f3a...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,1a730dd1844edc082bd852f6a8617db09885158f30e105...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...
2191,11be570f418641c06fe7a7bb832b237a51d3a6f1615f3a...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,3cbf33b87b930b9f7861e16e41b2596d98ddb08c128afb...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...
2192,11be570f418641c06fe7a7bb832b237a51d3a6f1615f3a...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,860b9434133c78fede874d0380cf50697b49af8c7c0b8d...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...
2193,11be570f418641c06fe7a7bb832b237a51d3a6f1615f3a...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,c4a161d608769cd6fe604685c9204f81351eb9f07032a7...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...


In [9]:
Communities = pd.read_csv('./Best_Clusters_DF.csv')
Communities.sort_values(by='cluster_size',inplace = True)

Communities.reset_index(drop = True, inplace = True)

def replaceSlash(row):
    return row.replace('\\n','\n')

repS = np.vectorize(replaceSlash)

Communities.cluster_authors = repS(Communities.cluster_authors)

Communities

,cluster_authors,cluster_size
0,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,2
1,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,2
2,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,2
3,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,2
4,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,2
...,...,...
66,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,35
67,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,45
68,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,50
69,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,82


In [10]:
Communities.cluster_size.sum()

871

In [11]:
Communities.index.values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70], dtype=int64)

In [12]:
#TEST FATTO IGNORANDO I WEIGHT TRA LE COMMUNITIES, CONSIDERO SOLO I #HOP

num_nodi = len(Communities)


#GRAPH CREATION


#STAMPA GRAFO
D=nx.DiGraph()

for i in Communities.index.values:
    D.add_node(str(i),nodeId = str(i))
    
for i in Communities.index.values: 
    
    inner_authors = Communities.iloc[i]['cluster_authors'].split("*")
    
    for j in Communities.index.values:
        
        if(i!=j):
        
            outer_authors = Communities.iloc[j]['cluster_authors'].split("*")
            
            temp_df = DB_AA_originale[DB_AA_originale['From_Author_Pub_Key'].isin(inner_authors)]
            
            temp_df = temp_df[temp_df['To_Author_Pub_key'].isin(outer_authors)]

            if(len(temp_df) > 0):           

                D.add_weighted_edges_from([(str(i),str(j),len(temp_df))])
            

D ha tanti nodi quante communities ed esiste un arco tra due communities A-B <=> almeno un autore di community A cita almeno un autore della community B

In [32]:
Communities_splitted = pd.DataFrame(columns=['community','authors'])
Communities_splitted.community = range(0,len(Communities))
Communities_splitted.set_index('community',inplace = True)

lun = 0

for i in range(len(Communities)):
    autori = Communities.loc[i]['cluster_authors'].split("*")
    Communities_splitted.loc[i]['authors'] = autori
    lun += len(autori)
    
print(lun)
    
Communities_splitted.head()

871


,authors
community,
0,[-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9...
1,[-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9...
2,[-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9...
3,[-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9...
4,[-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9...


In [31]:
Communities_splitted.iloc[0]['authors']

['-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA0dnd/2q1miceFJ4M/Hjo\nUGblGQjv0Slq9wPjlJpjD0qMzHFyct46l+gqfRYxVtoFEsQRqTLfj8CMcjJvn439\nxMlXByjnCXpI/y+SYHQNK+4STV/MnO5pr01B1ZDerHrBFcQu/IdKMhXwVBCb5gwP\nlzCeiP9Shw4I40rbU1zs593SBEPvQry681j210mbwge0TfkjDc4qCFJT5zACiQrB\nJzG86oankBgh/2X+Grlm8T4LlmVjvOajFAZsTwZta7HK3JX/a2z+OIdaaRwYwbkW\nQ17irPSo79E+Mczw7MzxRIIsegWGH/HzKRmgBLbVEN9fciAXcMoUzZuSRmjCKMwM\nHQIDAQAB\n-----END PUBLIC KEY-----\n',
 '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAtdv2ONOW6Csv8p1BK2Cg\nrQDP8koxz7h1ka2T/zw7opKQyyQljw63XCXwSVVKS5EtSKrBuQPWkM38uN98pvsW\nPevxUHBW+aueCjkLJBg46v7Ttu4NV0tOVMQT+ehyCQSzgma5guR3HO5QzrolwBK1\n9bSL4+8Tu5v993vQzuAKFuL7cP0WaD1iigmwCeP8JU6tBtX1HJ2Pi7IilGNC0q4T\nT1rAbSgmwqTGwWIVI52rMEbaddvhMyhdnQmg0XVrkRXX1dLe4skBCgCFz0U+YV63\nBlycRVrl7iFPJgOW/I4HRL+716Yg2Cldjo9Gifod/mGeyV8qlPBVkRStJsHFKWuc\nXQIDAQAB\n-----END PUBLIC KEY-----\n']

In [14]:
stima_veridicita_articoli = pd.DataFrame(columns=['article_id','estimate'])
stima_veridicita_articoli['article_id'] = msg_DB
stima_veridicita_articoli.set_index('article_id',inplace = True)
stima_veridicita_articoli

,estimate
article_id,
bea8b41266f68383aab0ab6f11e1423db72df15ff8357a1a769b5f4005b39533,NaN
48f59e57815078456b89fe34932606fcf1e5fa7874cf00498999ed99247a73b1,NaN
e2e9da947c52e0f6107d5aa5d6e03e0e9fcafe7bccde08b259ca41eb31c8a358,NaN
f2b1c07718734fb4f48789faef537bc69604b53befe3402c7fe4cfd0f71ededb,NaN
c356dd48fe8a94bc92ef446443ee88f49eb28dee5e95bd334c2dfd039e019a4d,NaN
...,...
642acd3957e86003df21148bec1f62cd6ecce16c7bab00fc9fcc5e766a0a2886,NaN
a90d93063f81d04b7b2b2a7ad70aa0a4ae1900c05d5328ecc5f1da486cbce44e,NaN
f1cb6682fae35ace95e977735bbe2976c10c1a77ed3b877b56aefc8a65ee9519,NaN


a[PR(auth_i)/PR_Max] + b[#cit_entranti_i/#cit_entranti_Max] + c[AVG(PR_auth_citano_i)/PR_Max] + d[Communities value]

In [15]:
#max_pr
PR_Max = PR_df.query('PR_values == PR_values.max()')['PR_values'][0]
print(PR_Max)

#max_cit_entranti
res = DB_AA_originale_senza_genesis.groupby('To_Author_Pub_key')['From_Author_Pub_Key'].count()
cit_entranti_max = res.max()
print(cit_entranti_max)

#max_len_#_hop
path = dict(nx.all_pairs_dijkstra_path(D))
max_len = 0
for node in D.nodes:
    for paths in path[node]:
        local_len = len(path[node][paths])
        if(local_len > max_len):
            max_len = local_len
            
print(max_len)

a = 0.25
b = 0.25
c = 0.25
d = 0.25

0.0424140089724826
58
6


In [33]:
stima = []

#for i in msg_DB:
 
article_id = msg_DB[10]

print(article_id)

autore = DB_AA_originale[DB_AA_originale['FromNodeId']==article_id]['From_Author_Pub_Key'].values[0]
PR_autore = PR_df[PR_df['Author']==autore]['PR_values'].values[0]
num_cit_entranti = len(DB_AA_originale[DB_AA_originale['To_Author_Pub_key']==autore])
autori_citanti = DB_AA_originale[DB_AA_originale['To_Author_Pub_key']==autore]['From_Author_Pub_Key'].values
AVG_PR_autori_citanti = PR_df[PR_df['Author'].isin(autori_citanti)]['PR_values'].values.mean()

#print(autori_citanti)

#calcolo la distanza tra le communities che mi citano

communities_citanti = []

for i in autori_citanti:
    print(i)
    for j in range(len(Communities_splitted)):
        if(i in Communities_splitted.loc[j]['authors'] ):
            print(j)
            communities_citanti.append(j)
communities_citanti

e8cfef1171e4bb3be7c1280ce56f973c1ab7d68aca78ef8bf04a26e6555ead35
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAt5DrRez/Tw2N2vID7pW6
O4SBE1pX/kj6v1IPwaGGnBCkBfZALCccG2GA+Q4qHov8pc/Mfx6ki0rT/81OuG4U
Hci7849B6KgqTobXSsmlOk3P4m2wk9TZY1la4M6vEUJFX/iF5Ws2J2oRZ8tinuH7
k17aTcOdUsmgnRlMuowgdZLR0GVzkCQz66eZitNjdrs1lXP6TsoRrMiz0pybPzxM
e/x2B4MoRMLKR88eBQCbRhyqNq+jnqlKGKvCNCy2qV8NIYfXaEDMdlgSISXR8m+d
4mekJ1+RfPgcbwSgONcKS7B1ykHOro2eW1AXJpCVt4SJIzETDCDNrpBUSH99WMmH
lwIDAQAB
-----END PUBLIC KEY-----

68
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAwpTIFFcInXNkGLcwnraG
xwTDHAtSGxsvloN+8JHLEYk7Lzkix49quk1DeCKmU8V5tgxOmuiEHbz8SzFdwLdS
2yx6eL9JJhwiBpj5LueOI5lrxb1T7kLIWvsKGN0XKwopTLYMD5Xg+fzboVr7gRJ2
7hjonXTat5FAb2DjHUSKQAayXF/eBwS8DNbUWTUNgZbH4Mgako8VF4UKHvWb91ip
aQ8432X47zuAydd0/KyxTzk2a9GywrjqN/wK6nYIS4QUsvTT8wuVotY6EPqjuiy9
cjrxd2ch9xmCw7COLBBwjGqGzsq6HHlQIFLzjskSjxaAVRc7vaitesl5V8M76CcQ
JwIDAQAB
-----END PUBLIC KEY-----

68
-----BEGIN PUBLIC KEY----

[68, 68, 70, 68]

In [168]:
#DB_AA_originale[DB_AA_originale['From_Author_Pub_Key']==autori_citanti[2]]

In [169]:
#DB_AA_originale[DB_AA_originale['To_Author_Pub_key']==autori_citanti[2]]

In [170]:
#DB_AA_originale.iloc[1901]['From_Author_Pub_Key'] == DB_AA_originale.iloc[1901]['To_Author_Pub_key']

In [131]:
PR_df[PR_df['Author']==autori_citanti[2]]

,Author,PR_values
530,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,0.000552


In [112]:
len(Communities_splitted.loc[68]['authors'])

50

In [174]:


for j in range(len(Communities_splitted)):
    if(autori_citanti[2] in Communities_splitted.loc[j]['authors'] ):
        print(j)

In [171]:
autori_citanti[2]

'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA7rOk3oVB8+h+NkTo0ZNg\n6Nc1OiKhfCVYBSxJwyVMQcOGd6/NJXXOPD0N+UTDR3kKakZD47AO2G28p2CBNcdm\nIju29ZYUNi6c9/uWBkLHYqM7PTPGwNXxvCK7RgzfZr7sGuohzaE+k3IWWzjnk9VF\nzpU1+CqUhOkjdjEbAqNqWkVlJojhf07Ic/hf1FP2yAkE+Xn6l83T10+i8G6+Y1NW\ngoKDYFlbQtuplTi9nact1h6nkqKhN/w2njahdWJY7xSu/VNRpytJb3aXV5SyTk/J\nJZd2JyryXl60/1m8qBg9xF5zReOjGGq4JU18QTB74n582qYfMStSs3XNAMt3w3Y/\neQIDAQAB\n-----END PUBLIC KEY-----\n'

In [176]:
DF = pd.read_csv('./AM_Matrix.csv')
DF['FROM'] = DF.columns
DF.set_index('FROM', inplace = True)
for i in DF.columns.values: 
        if(DF.loc[i].sum()==0 and DF[i].sum()==0):
            DF.drop(index = i, inplace=True)
            DF.drop([i], axis = 1, inplace=True)
DF

,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA+S7JOUZTJ4ATDakYy4sq\nhaHyp7cOryi/bXk7a1AM/Ypetd1lAjvEfBXzriLeeVzffOloBJYYMEYFUlxoMqIA\nyDjtVHt0c9F9vOl7w68MlDPLFqdDWD+rTrXbqdQSa69PbyPw8pRH9OhlV1jS216N\n6I75w0sRUZTmYTp4stxg2wOv6+IrD+GGtoNvKayD6o3wf+/6lN2PANvwQR+wXmxE\ndfKf6aT9K+bWl0+yOccujbfz5buLEfS68Yt90MJmxPtrd8YA7y2uUI5K09/eP3SX\nrpulxZ6zlRkn8aPhC5/lVOptGopMiz66BDM01GriO0U4iMtQiE3n21uyCWzP6I/T\nYQIDAQAB\n-----END PUBLIC KEY-----\n,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA/UfntDOIilbHbUZx9vLr\nH+YaSTywCXEpgml5GcvFrxAh2Rs0iydOhxKXasqgUI/OaXYaPaO9xYVtirBaPMYB\nBGBsRyBUDrjzxBFLJXBsag2RAg0E6YExoVh9GOEzhuNEdC/VEj7ARHe6e7PbLbFu\ng/ZRQ0zI3ao0pETR/WDXqjhmBk8fLIlap358MGYZyOTW3axboMZANbEtfRtt+xj1\n3Ye/zINIPJvS84vuH/hShofkI6ijz/OpJ1lIEw+MBcqMp1LUPypFuU1VQ5y/j2F9\nAPcJanKYlDpGV5bdwCWZEYB6vqNb7uQmBtpqVtCSEFqk3hPmJS+ejuQrQ8km9dy7\ngwIDAQAB\n-----END PUBLIC KEY-----\n,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA007iPtuOQbXOsaBTsDOp\n7k7G6DrA63ceO09yX0HtEPMhSIUmrsagaJv61U9RmZLhKNvfKRjdFR+XUvZT4ZL6\nwmn6tiA7oxsC+lzpnH1qHnELSNk/oDtwQ3WrGKFdkforuJxA4RG40mlvL2q3YMGg\nnU41rR6sfEkTZJ5SXCUhR9Rb4DRcANC3neDiyeGpEY15QY5MNgqunOrZU+Uf8H95\nujd9rlKE6mbW+FsgoQ4R5NvYAAydfgsgo440d56/joVU7qY0p5Njtv3tSnAm5HxY\nDIxMoz1Vrmj0xZA/aYLfQ0TPkNJ10+l41OiF8xwH8doWyaJ4zevGj0wt1bDLOg43\nnQIDAQAB\n-----END PUBLIC KEY-----\n,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA03t2yD6cuBFBjyGU2Qq8\nfbFRMFggaTX6Cw41TD7OvO6Ur00+KAPEpKXzCT2qmRDuJERYsGaCThXu76SsxCqO\nYkDTCnTpAgVJhYFTaQCi7Cckg4gIJBk21fwBieOqkHMs5JFFV6N0At6h7aGxpnN2\n5lhf+6ghh5RcvCQmAyYMZ7jyeM5ZyhoA6yb7oz/Atr+BHjA23Le0iJDjNrsqADWI\nPZkdgqjn7BQk8kGhukKwmu0zkwuTah+6gPjkWNmrhG2N/9BYOfrDlQxc748zg0Fv\nc4DQhKjeeGXAc95RAc57VZ+zrFtQKDLWMLMlcSUii8CVb7zdRYFGW+Z/+G+1yPBz\nGwIDAQAB\n-----END PUBLIC KEY-----\n,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA04JGxt6hWfNhv0m4BLDR\nPOjtbzCdUJMqFBo3T59SqGx9QWj92ro5SE2AgawOjSNLcdJiURtaOfJJFsAaCe7B\nBu/Byui9+vmdJjTV0nFaVy5zMLKibAjf9BERos9bOTvGSomfdaP/gLPuY6yXDUmb\nbgnKx0fPNZlIc1cBkPXbh+8jhfV41c39991lWbbW2WXDL52zsEUyvhVsU5lYsDEG\n4INQHvMzC001RSgF1n190lFWwg2QDBqwVbhjYg9ugkPuOtjR4vyVKkktF2Xl2Xje\npwKuCCDLgYA6j8XUnmUQh0qT/wtVObs/LYI40VwrTXH85/pT+nB7cJqVZFi3Gtwq\nuQIDAQAB\n-----END PUBLIC KEY-----\n,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA06moZ+ZMk4Aorxo8P8Ip\nyp5WW3J9C8gxvYnogvMsitJLWbkH/ZEELCGB4w3R5MVwIHhfFVOER+SUcgwPWxJK\npwXLFimmig309VR8JKUGJUJtUwbDcx7qTF201PnjHC2UAXB+yrqBvQoruJaI3gnJ\noKE50P9gi0Sox3ubEp6cA+I1Wq7cFcufKIgZ8KqUdN8gHQCQEtV+rPI1q9ci7Nk1\nU0YAdBJyOfKnYEE4bzZllCAJ3lg9bppkRu1FiR25syLoQ/PIlBrxwAIGLd+DJMR7\nOpc+oOUQ+Tufr0A/H64H8ST4AQG8UvjShzdYZnM2nOtdOe9g5jY1+X4e0gQ06vJ+\ntQIDAQAB\n-----END PUBLIC KEY-----\n,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA07aFAf9mlsMND9TX+ZC9\nx7GUpKMogXxk9PNZf2C58NEZmc0TLw7EsDK/Nzo5LJJ27BrabePWOf7lLO+qUzNF\njXNzmJOVnO9ivUQqlHKiodcO8v2HERUax8ZCwr8vok4eAQwp5jMHSAyADYjuj8Sm\nRbfRbJqAKsOLCsNnJ7+bWW07ZVVF/c/X3oKRx8iU9IFE6YIZlxNdm07qT2SMhsYQ\nFVAVqdAoLosBlcR0+r7wGGxKEAC4WChsa6JvPb/1BbElA+/jrVS2+MxZEAKqN33v\nvtjFXhCzycGCeMC3uqoroiR3hjt7Ni7epKn8fhtizKZ+Ai6mNPozF5Fw+3Uk4Gsq\n4wIDAQAB\n-----END PUBLIC KEY-----\n,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA08Yrh7iwqx8bxY0NDc4y\n2AbZJMeIy1zi5MI51XLbo//xC6SRrK0XcN/8dCgetThPnonu4n5ZIXM1TyM+ryP6\nmiQh4L3TM3OqhywpSIZNpOzKQ/by2XX5idupPlieyA4ugkSAmqp/KFsr5/yapL1v\nPaP7fKs6FxVeQ4iWFLdpUkDnytl9vRRPWOhLhnX1R+B4q3ut/aI//PGkaZpHqsQ9\n4f1ksfibjIYtRYBMYn84CkvTOE7e1edvMznuZxOJT2GgouEaKeEVqE2GAK4kMphg\nvPmDi74SOZlS2DP9qWWdmULRgPeCdfLhNMSmX9kC1inYini0qztxH3+k4rZ1mWgy\nAQIDAQAB\n-----END PUBLIC KEY-----\n,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA0A00nQoyb0o3eS3ZcbX9\nGzVjCPRg2ksCAmbJkbznebCABzIbjpx5vO0S8IbBUEo7JElJyzYbMmIC+I9aAzoP\nrmBew/oH8Q/2MHxXxXuBEXjI/IZ8tgynwVr+gMITlbmLhmBhYNsisxHkfPUBMirY\nIlFgBRcml78MHpGHCvOmQFKA1jB0hYj75bBXpY5fkJZJvJKwuFG1QSRlH7fNx7og\nMvoNylkSNEGjyZt9RO1

In [167]:
for i in DF.columns.values: 
    if(DF.loc[i].sum()==0):
        if(DF[i].sum()==0):
            print(i == autori_citanti[2])

False
False
False
False
False
False
False


In [177]:
autori_citanti[2] in DF.columns.values

True

In [145]:
autori = np.unique(DB_AA_originale.From_Author_Pub_Key.values)
len(autori)

878

In [18]:
autori_distinti_1 = []
for i in range(len(Communities)):
    for j in Communities.loc[i]['cluster_authors'].split('*'):
        autori_distinti_1.append(j)
    

autori_distinti_1 = np.unique(autori_distinti_1)
len(autori_distinti_1)


871

In [19]:
autori_distinti2 = []
for i in range(len(Communities_splitted)):
    for j in Communities_splitted.loc[i]['authors']:
        autori_distinti2.append(j)
    

autori_distinti2 = np.unique(autori_distinti2)
len(autori_distinti2)

612

In [27]:
autori_citanti[0] in autori_distinti2

True